# Exercise 8A - NSGA-II Algorithm

Introduced in Tutorial 8

Demonstrate how to implement the NSGA-II algorithm in Python using the pymoo package. The problem from Exercise 4B will be re-used as an example.

### Colour codes

<span style="color:orange;"> Orange text is for emphasis and definitions </span>

<span style="color:lime;"> Green text is for tasks to be completed by the student </span>

<span style="color:dodgermagenta;"> Blue text is for Python coding tricks and references </span>

## Load all the necessary Python packages
All packages should work with Conda environment if installed on your machine. Otherwise all necessary packages can be installed in a virtual environment (.venv) in VS Code using: Ctrl+Shift+P > Python: Create Environment > Venv > Python 3.12.x > requirements.txt

<span style="color:orange;"> NOTE: that we are using the *pymoo* package will be used. You may need to install this package using pip.</span>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.selection.rnd import RandomSelection
from pymoo.termination import get_termination
from pymoo.optimize import minimize

## 1. Getting Started

### 1.1 Problem Definition
Define the two objective functions to be used in this exercise. The functions are the same as those used in Exercise 4B.


In [ ]:
def objectiveFunction_1(x_1, x_2):
    return 3 * (1 - x_1) ** 2 * np.exp(-x_1 ** 2 - (x_2 + 1)**2) - 10 * (x_1 / 5 - x_1 ** 3 - x_2 ** 5) * np.exp(-(x_1) ** 2 - (x_2 **2)) - 3 * np.exp (-( x_1 + 2)**2 - x_2 **2) + 0.5 * (x_1 + x_2)

def objectiveFunction_2(x_1, x_2):
    return 2 * (1 + x_2) ** 2 * np.exp((-x_2 ** 2) - (x_1)**2) - 10 * (-x_2 / 5 + x_2 ** 3  + x_1 ** 5) * np.exp(-(x_2 ** 2) - (x_1) **2) - 2 * np.exp (- (2 - x_2)**2 - x_1 **2)
   

## 2. Setting up Pymoo

Pymoo (Python Multi-objective Optimization) is a package which implements several optimization algorithms including the Particle Swarm and Genetic Algorithm discussed in the lectures. Instructions on the capabilities of pymoo and its various algorithms, plus examples are located here [Link](https://pymoo.org/). A step-by-step example of how to set-up a basic problem with pymoo is demonstrated here.

### 2.1 Setting up the Problem class

The first step is to define the problem. The problem essentially describes the calculation which is to be done and the outputs which are passed to the optimization algorithm. Pymoo requires this problem to be set up in a *class*. A class combines data and functions together in one context. More information on classes is here [GeeksForGeeks](https://www.geeksforgeeks.org/python/python-classes-and-objects/). The choice of whether to use a function or a class is dependant on the situation and the package author's preference. Here the authors of pymoo have chosen to use classes to define problems so we must work with this structure.

The following is a simple example of the classes we will be using where we take a base number (*x*) and calculate its exponent. This class has two <span style="color:orange;">methods (defs)</span>: one to initiate the class (\_\_init\_\_), and another to evaluate the functions and return the results (\_evaluate).

In [ ]:
class Example():
    def __init__(self, x):
        """ 
        Initiate the function and create the attribute for the base x.
        Note the we use the 'self' variable to assign attributes to this class
        """
        self.x = x
    def _evaluate(self, y):
        """ 
        Perform the calculation and return the results
        """
        return self.x ** y


Using this class.

In [ ]:
# Create instance for base 3
x = 3
A = Example(x)
# Access the attribute x
print (f"base = {A.x}")

# Then use the evaluate method of this instance
ans  = A._evaluate(2)
print (f"ans = {ans}")

This exercise's Pymoo problem looks like this. <span style="color:orange;">Some of the details are complex with regards to class inheritance and aren't necessary to understand for the coursework.</span> This class has two <span style="color:orange;">methods (defs)</span>: one to initiate the class (\_\_init\_\_), and another to evaluate the functions and return the results (\_evaluate).

1. We define a new class named *Problem_4B*. This class inherits attributes and methods from the *ElementwiseProblem* parent class.
2. In the \_\_init\_\_ statement we have a super().\_\_init\_\_ statement which tells us which attributes we want to inherit from the parent class.
    * These are explained in the next code cell
    * Note that they are also default arguments which can change when we create an instance of this class
3. Define the evaluate method where we call each of the objective functions and return the results to the Pymoo output dictionary.


In [ ]:
class Problem_4B(ElementwiseProblem):
    # Initialise the class with these default arguments
    def __init__(
            self,
            n_var = 2,
            n_obj = 2,
            n_ieq_constr = 0,
            xl = np.array([-3, -3]),
            xu = np.array([3, 3])
        ): 

        #Inherit attributes and methods from the parent class Elementwise using super()
        super().__init__()

        # Assign arguments as attributes
        self.n_var = n_var                  # Number of input variables (x)
        self.n_obj = n_obj                  # Number of objective functions
        self.n_ieq_constr = n_ieq_constr    # Number of inequality constraints, which we won't be using for this problem or the next
        self.xl = xl                        # Lower bounds of each variable
        self.xu = xu                        # Upper bounds of each variable


    def _evaluate(self, x, out): 
        """
        Evaluate the both objective functions
        x = the input variables
        out = is pymoo's output dictionary which will be assigned the results to ["F"]
        """
        f1 = objectiveFunction_1(x[0], x[1])
        f2 = objectiveFunction_2(x[0], x[1])

        # Return the results of each objective to the output dictionary
        out["F"] = np.array([f1, f2])

Create an instance of the problem using the default arguments defined above

In [ ]:
problem = Problem_4B()

We can access attributes from the problem just like we did for the example class.

In [ ]:
print (f"Lower bounds = {problem.xl}")
print (f"Upper bounds = {problem.xu}")

<span style="color:orange;">In summary, it is important to understand how to set-up and call a Python class when using Pymoo to solve multi-objective problems. You can follow the framework here or from the examples on Pymoo's website.</span>

### 2.2 Setting up the Algorithm
In this step we select which algorithm we want to use and set up the instructions for how that algorithm will work.

Here we are going to use the NSGA2 algorithm. Remember that pymoo implements a large number of algorithms including Particle Swarm and other variants of the Genetic Algorithm [List](https://pymoo.org/algorithms/list.html#nb-algorithms-list). The algorithm also needs a list of parameters to be passed:

* pop_size = The number of individuals to be evaluated
* n_offsprings = The number of offsprings to be created after each method. Note *pop_size* - *n_offsprings* = to the number of individuals who survive into this generation.
* sampling = How the initial sample is determined. Options are *FloatRandomSampling* and *LHS*
* selection = Defines how parents are chosen for the crossover stage. Options are *RandomSelection* and *TournamentSelction*.
* crossover = Defines the crossover algorithm to be followed. Here we are using Simlated Binary Crossover (SBX) with its default parameters, but there others which we can select.
* mutation = Defines the mutation model we wish to use with the probability of mutation (&eta;). Options are *Polynomial Mutation* (PM) and *Bitflip Mutation* (BM).
* eliminate_duplicates = Removes any duplicates from the populations before evalauting

Note that you need to import all of these models. See import cell


In [ ]:
algorithm = NSGA2(
    pop_size = 30,
    n_offsprings = 15, # Must be less than the population size
    sampling = FloatRandomSampling(),
    selection = RandomSelection(),
    crossover = SBX(),
    mutation = PM(eta=20),
    eliminate_duplicates = True
)


### 2.3 Setting up the termination criteria.
These are the instructions for termination of the algorithm. Pymoo has default settings for termination based on design space tolerance, objective space tolerance, number of generations and time [Link](https://pymoo.org/interface/termination.html?highlight=get_termination).
Here we will instruct it to end after 20 generations. You may also set limits based on the number of evaluations or time as shown in the commented code. 

For the coursework, setting a restriction based on execution time may be the most valuable for you.

In [ ]:
termination = get_termination("n_gen", 30)
#termination = get_termination("n_eval", 500)
#termination = get_termination("time", "00:00:05")

### 2.4 Running the algorithm as a minimization program

Use the minimize function to run the algorithm. Here we pass the pre-defined problem, algorithm, and termination criteria, as well as an (optional) random_seed. We will also set save_history to true to store the intermediate results, and verbose to True to generate updates as the algorithm progresses.

In [ ]:
res = minimize(problem,
               algorithm,
               termination,
               seed = 1,
               save_history = True,
               verbose = True)


The output above contains the following information:
* n_gen =  number of generations
* n_eval = number of evaluations done
* n_nds =  number of non-dominated solutions
* eps = a statistic which measures the relative movement of the ideal solution v. the nadir solution at each generation. This metric should be converging toward zero with each run.
* indicator = whether this relative movement was strong toward the ideal or nadir solution or remained relatively stable (f)

<span style="color:lime;">In the example above, when can we say that a pareto front has started to emerge? </span>
<span style="color:lime;">Is our termination criteria adequate? </span>


### 2.5 Assessing the results

We can query the both the final design and objective spaces. The design space is the variable *X* for all individuals and the objective space *F* is the evaluated function for each individual. They are accessible throught the results *class*.

For the final iteration those values are:

In [ ]:
# Access the arrays for X and F the same way you would for a class attribute
X = res.X
F = res.F

print ("The first ten individual's of X coordinates")
print (X[:10])
print ("\nThe first ten individual's objective function values")
print (F[:10])

### 2.5.1 Comparison with Exercise 4B

For comparison, I have included an underlay of the results from Exercise 4B which used full-factorial grid-sampling

In [ ]:
xl, xu = problem.bounds()

fig, ax = plt.subplots(ncols = 2)

# Plot the first figure
img = plt.imread(Path("src", "images", "underlay_designSpace.png"))
ax[0].imshow(img, extent = [-3.5, 3.5, -3.5, 3.5], alpha = 0.5)

ax[0].scatter(X[:, 0], X[:, 1], s=30, facecolors='none', edgecolors='r')
ax[0].set_title("Design Space")

ax[0].set_aspect("equal", "box")

# Plot the objective space
img = plt.imread(Path("src", "images", "underlay_objectiveSpace.png"))
ax[1].imshow(img, extent = [-8, 9.25, -8.5, 8.5], alpha = 0.5)

ax[1].scatter(F[:, 0], F[:, 1], s=30, facecolors='none', edgecolors='r')
ax[1].set_title("Objective Space")

ax[1].set_aspect("equal", "box")

fig.set_figwidth(10)
fig.tight_layout()

plt.show()


Note theat the results of the NSGA-II algorithm correspond with what we found in Lecture 4B. Note there is some error in the scaling of the underlay.

### 2.5.2 Plotting the Progression of the Genetic Algorithm
The following plots show how the individuals evolve over time in terms of the design space (X) and objective space (F).

First for the design space:

In [ ]:
# Retrieve the values at specific generations from the results history using list comprehension techniques.
generations = [0, 5, 10, 15, 20, 30]

X_0 = np.array([res.history[1].pop[i].X for i in range(algorithm.pop_size)])
X_5 = np.array([res.history[5].pop[i].X for i in range(algorithm.pop_size)])
X_10 = np.array([res.history[10].pop[i].X for i in range(algorithm.pop_size)])
X_15 = np.array([res.history[15].pop[i].X for i in range(algorithm.pop_size)])
X_20 = np.array([res.history[20].pop[i].X for i in range(algorithm.pop_size)])

# Generate the plots, with correct limits and titles
xl, xu = problem.bounds()

fig, ax = plt.subplots(ncols = 3, nrows = 2)

# Draw scatter plots
ax[0, 0].scatter (X_0[:, 0], X_0[:, 1], s=30, facecolors='none', edgecolors='magenta',)
ax[0, 1].scatter (X_5[:, 0], X_5[:, 1], s=30, facecolors='none', edgecolors='magenta',)
ax[0, 2].scatter (X_10[:, 0], X_10[:, 1], s=30, facecolors='none', edgecolors='magenta',)
ax[1, 0].scatter (X_15[:, 0], X_15[:, 1], s=30, facecolors='none', edgecolors='magenta',)
ax[1, 1].scatter (X_20[:, 0], X_20[:, 1], s=30, facecolors='none', edgecolors='magenta',)
ax[1, 2].scatter (X[:, 0], X[:, 1], s=30, facecolors='none', edgecolors='magenta',)

# Set the axis and title for each graph in a for loop
for _ax, _generation in zip(ax.flat, generations):
    _ax.set_xlim(xl[0], xu[0])
    _ax.set_ylim(xl[1], xu[1])
    _ax.set_title(f"Generation {_generation}", fontsize=9)

fig.set_figheight(5)
fig.set_figwidth(8)
fig.tight_layout()
plt.show()

In [ ]:
# Retrieve the values at specific generations from the results history using list comprehension techniques.
F_0 = np.array([res.history[1].pop[i].F for i in range(algorithm.pop_size)])
F_5 = np.array([res.history[5].pop[i].F for i in range(algorithm.pop_size)])
F_10 = np.array([res.history[10].pop[i].F for i in range(algorithm.pop_size)])
F_15 = np.array([res.history[15].pop[i].F for i in range(algorithm.pop_size)])
F_20 = np.array([res.history[20].pop[i].F for i in range(algorithm.pop_size)])


# Generate the plots, with correct limits and titles
xl, xu = problem.bounds()

fig, ax = plt.subplots(ncols = 3, nrows = 2)

# Draw scatter plots
ax[0, 0].scatter (F_0[:, 0], F_0[:, 1], s=30, facecolors='none', edgecolors='blue',)
ax[0, 1].scatter (F_5[:, 0], F_5[:, 1], s=30, facecolors='none', edgecolors='blue',)
ax[0, 2].scatter (F_10[:, 0], F_10[:, 1], s=30, facecolors='none', edgecolors='blue',)
ax[1, 0].scatter (F_15[:, 0], F_15[:, 1], s=30, facecolors='none', edgecolors='blue',)
ax[1, 1].scatter (F_20[:, 0], F_20[:, 1], s=30, facecolors='none', edgecolors='blue',)
ax[1, 2].scatter (F[:, 0], F[:, 1], s=30, facecolors='none', edgecolors='blue',)

# Set the axis and title for each graph in a for loop
for _ax, _generation in zip(ax.flat, generations):
    _ax.set_xlim(-8, 4)
    _ax.set_ylim(-8, 8)
    _ax.set_title(f"Generation {_generation}", fontsize=9)

fig.set_figheight(5)
fig.set_figwidth(8)
fig.tight_layout()
plt.show()

## 3. Test Your Understanding
* <span style="color:lime;"> What do you think would happen if you reduced or increased the population size? What effect would it have on convergence and the termination criteria? WWould it lead to a difference in the shape of the resultant pareto front?</span>
* <span style="color:lime;">For Question 4 of Coursework 1 you were asked to analyze a tri-objective problem. What would be required of you to convert this example into a tri-objective problem.</span>
    * <span style="color:lime;">Consider what you would need to change inside the problem class definition.